# SQL 的五十道練習

> 子查詢

[數據交點](https://www.datainpoint.com) | 郭耀仁 <yaojenkuo@datainpoint.com>

## 練習題指引

- 由於近期 mybinder.org 的服務不穩定，新增 Google Colab 作為另一個寫作練習題的平台。
- 開始寫作之前，可以先按上方「Copy to Drive」按鈕將筆記本複製到自己的 Google 雲端硬碟。
- 在每份練習題的開始，都會先將四個學習資料庫載入環境。
- 因此 SQL 可以指定四個學習資料庫中的資料表，不需要額外指定資料庫。
- 在 SQL 語法起點與 SQL 語法終點這兩個單行註解之間撰寫能夠得到預期結果的 SQL。
- 可以先在自己電腦的 SQLiteStudio 或者 DBeaver 寫出跟預期結果相同的 SQL 後再複製貼上到練習題。
- 執行測試的方式為點選右上角 Connect，然後點選上方選單的 Runtime -> Restart and run all -> Yes -> Run anyway。
- 移動到 Google Colab 的最後一個儲存格看測試結果。
- 可以每寫一題就執行測試，也可以全部寫完再執行測試。
- 練習題閒置超過 10 分鐘會自動斷線，這時只要重新點選練習題連結即可重新啟動。

In [10]:
!wget -N https://raw.githubusercontent.com/datainpoint/classroom-hahow-sqlfifty/main/databases/nba.db
!wget -N https://raw.githubusercontent.com/datainpoint/classroom-hahow-sqlfifty/main/databases/covid19.db
!wget -N https://raw.githubusercontent.com/datainpoint/classroom-hahow-sqlfifty/main/databases/twElection2020.db
!wget -N https://raw.githubusercontent.com/datainpoint/classroom-hahow-sqlfifty/main/databases/imdb.db
!wget -N https://raw.githubusercontent.com/datainpoint/classroom-hahow-sqlfifty/main/exercise_index.json

--2026-01-07 12:56:54--  https://raw.githubusercontent.com/datainpoint/classroom-hahow-sqlfifty/main/databases/nba.db
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 233472 (228K) [application/octet-stream]
Saving to: ‘nba.db’

nba.db              100%[===================>] 228.00K  --.-KB/s    in 0.02s   

Last-modified header missing -- time-stamps turned off.
2026-01-07 12:56:54 (14.6 MB/s) - ‘nba.db’ saved [233472/233472]

--2026-01-07 12:56:54--  https://raw.githubusercontent.com/datainpoint/classroom-hahow-sqlfifty/main/databases/covid19.db
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... c

In [11]:
import sqlite3
import unittest
import json
import os
import numpy as np
import pandas as pd
conn = sqlite3.connect('nba.db')
conn.execute("""ATTACH 'covid19.db' AS covid19""")
conn.execute("""ATTACH 'twElection2020.db' AS twElection2020""")
conn.execute("""ATTACH 'imdb.db' AS imdb""")

## 30. 從 `nba` 資料庫的 `players` 資料表運用子查詢找出 NBA 中身高最高與最矮的球員是誰，參考下列的預期查詢結果。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(5, 3) 的查詢結果。

```
  firstName  lastName  heightMeters
0     Tacko      Fall          2.26
1     Jared    Harper          1.78
2   Tremont    Waters          1.78
3    Markus    Howard          1.78
4   Facundo  Campazzo          1.78
```

In [12]:
find_tallest_shortest_from_players =\
"""
-- SQL 查詢語法起點
SELECT firstName,
       lastName,
       heightMeters
  FROM players
 WHERE heightMeters = (
                          SELECT MAX(heightMeters)
                            FROM players
                      )
       OR
       heightMeters = (
                          SELECT MIN(heightMeters)
                            FROM players
                      );
-- SQL 查詢語法終點
"""

## 31. 從 `nba` 資料庫的 `players` 資料表運用子查詢計算球員的國籍佔比，參考下列的預期查詢結果。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(43, 2) 的查詢結果。

```
                   country  player_percentage
0                      USA           0.764463
1                   Canada           0.035124
2                   France           0.022727
3                Australia           0.018595
4                  Germany           0.012397
5                   Serbia           0.012397
6                  Croatia           0.008264
7                    Spain           0.008264
8                   Turkey           0.008264
9                   Greece           0.006198
10                   Italy           0.006198
11                  Latvia           0.006198
12               Lithuania           0.006198
13                 Nigeria           0.006198
14                 Senegal           0.006198
15                Slovenia           0.006198
16                 Bahamas           0.004132
17                  Brazil           0.004132
18                Cameroon           0.004132
19                   Japan           0.004132
20                 Ukraine           0.004132
21                  Angola           0.002066
22               Argentina           0.002066
23                 Austria           0.002066
24  Bosnia and Herzegovina           0.002066
25          Czech Republic           0.002066
26                     DRC           0.002066
27      Dominican Republic           0.002066
28                   Egypt           0.002066
29                 Finland           0.002066
30                   Gabon           0.002066
31                 Georgia           0.002066
32                  Guinea           0.002066
33                  Israel           0.002066
34                 Jamaica           0.002066
35              Montenegro           0.002066
36             New Zealand           0.002066
37   Republic of the Congo           0.002066
38             Saint Lucia           0.002066
39             South Sudan           0.002066
40                   Sudan           0.002066
41             Switzerland           0.002066
42          United Kingdom           0.002066
```

In [13]:
calculate_player_percentage_from_players =\
"""
-- SQL 查詢語法起點
SELECT COUNTRY,
       ROUND(COUNT( * ) * 1.0 / (
                                    SELECT COUNT( * )
                                      FROM Players
                                ), 6) AS player_percentage
  FROM Players
 GROUP BY COUNTRY
 ORDER BY COUNT( * ) DESC;
-- SQL 查詢語法終點
"""

## 32. 從 `nba` 資料庫運用子查詢找出 NBA 的場均得分王（`ppg`），參考下列的預期查詢結果。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(1, 2) 的查詢結果。

```
  firstName lastName
0     Kevin   Durant
```

In [14]:
find_highest_ppg_player_from_nba =\
"""
-- SQL 查詢語法起點
SELECT firstName,
       lastName
  FROM players
 WHERE personID = (
                      SELECT personID
                        FROM career_summaries
                       WHERE ppg = (
                                       SELECT MAX(PPG)
                                         FROM career_summaries
                                   )
                  );
-- SQL 查詢語法終點
"""

## 33. 從 `nba` 資料庫運用子查詢找出目前布魯克林籃網隊（Brooklyn Nets）的球員名單，參考下列的預期查詢結果。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(17, 2) 的查詢結果。

```
   firstName         lastName
0   LaMarcus         Aldridge
1      Kevin           Durant
2       Jeff            Green
3    DeAndre           Jordan
4      Blake          Griffin
5      James           Harden
6      Kyrie           Irving
7    Spencer        Dinwiddie
8        Joe           Harris
9      Tyler          Johnson
10   Timothe  Luwawu-Cabarrot
11     Bruce            Brown
12     Alize          Johnson
13    Landry           Shamet
14     Chris          Chiozza
15    Reggie            Perry
16   Nicolas          Claxton
```

In [15]:
find_brooklyn_nets_roster_from_nba =\
"""
-- SQL 查詢語法起點

-- SQL 查詢語法終點
"""

## 34. 從 `twElection2020` 資料庫的 `presidential` 資料表計算各組候選人的得票率，參考下列的預期查詢結果。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(3, 2) 的查詢結果。

```
   candidate_id votes_percentage
0             1            4.26%
1             2           38.61%
2             3           57.13%
```

In [16]:
calculate_president_votes_percentage_from_twelection2020 =\
"""
-- SQL 查詢語法起點

-- SQL 查詢語法終點
"""

## 執行測試！

Runtime -> Restart and run all -> Yes -> Run anyway.

In [17]:
class TestSubQuery(unittest.TestCase):
    def test_30_find_tallest_shortest_from_players(self):
        tallest_shortest_from_players = pd.read_sql(find_tallest_shortest_from_players, conn)
        self.assertEqual(tallest_shortest_from_players.shape, (5, 3))
        column_values = set(tallest_shortest_from_players.iloc[:, 2].values)
        self.assertTrue(2.26 in column_values)
        self.assertTrue(1.78 in column_values)
    def test_31_calculate_player_percentage_from_players(self):
        player_percentage_from_players = pd.read_sql(calculate_player_percentage_from_players, conn)
        self.assertEqual(player_percentage_from_players.shape, (43, 2))
        column_values = set(player_percentage_from_players.iloc[:, 0].values)
        self.assertTrue('USA' in column_values)
        self.assertTrue('Canada' in column_values)
        self.assertTrue('France' in column_values)
    def test_32_find_highest_ppg_player_from_nba(self):
        highest_ppg_player_from_nba = pd.read_sql(find_highest_ppg_player_from_nba, conn)
        self.assertEqual(highest_ppg_player_from_nba.shape, (1, 2))
        row_values = set(highest_ppg_player_from_nba.iloc[0, :].values)
        self.assertTrue('Kevin' in row_values)
        self.assertTrue('Durant' in row_values)
    def test_33_find_brooklyn_nets_roster_from_nba(self):
        brooklyn_nets_roster_from_nba = pd.read_sql(find_brooklyn_nets_roster_from_nba, conn)
        self.assertEqual(brooklyn_nets_roster_from_nba.shape, (17, 2))
        first_names = set(brooklyn_nets_roster_from_nba.iloc[:, 0].values)
        last_names = set(brooklyn_nets_roster_from_nba.iloc[:, 1].values)
        self.assertTrue('Kevin' in first_names)
        self.assertTrue('James' in first_names)
        self.assertTrue('Kyrie' in first_names)
        self.assertTrue('Blake' in first_names)
        self.assertTrue('LaMarcus' in first_names)
        self.assertTrue('Durant' in last_names)
        self.assertTrue('Harden' in last_names)
        self.assertTrue('Irving' in last_names)
        self.assertTrue('Griffin' in last_names)
        self.assertTrue('Aldridge' in last_names)
    def test_34_calculate_president_votes_percentage_from_twelection2020(self):
        president_votes_percentage_from_twelection2020 = pd.read_sql(calculate_president_votes_percentage_from_twelection2020, conn)
        self.assertEqual(president_votes_percentage_from_twelection2020.shape, (3, 2))
        np.testing.assert_equal(president_votes_percentage_from_twelection2020.iloc[:, 1].values,
                               np.array(['4.26%', '38.61%', '57.13%']))

suite = unittest.TestLoader().loadTestsFromTestCase(TestSubQuery)
runner = unittest.TextTestRunner(verbosity=2)
test_results = runner.run(suite)
number_of_failures = len(test_results.failures)
number_of_errors = len(test_results.errors)
number_of_test_runs = test_results.testsRun
number_of_successes = number_of_test_runs - (number_of_failures + number_of_errors)
#cwd = os.getcwd()
folder_name = "10-subqueries"
with open("exercise_index.json", "r") as content:
    exercise_index = json.load(content)
chapter_name = exercise_index[folder_name]

test_30_find_tallest_shortest_from_players (__main__.TestSubQuery.test_30_find_tallest_shortest_from_players) ... ok
test_31_calculate_player_percentage_from_players (__main__.TestSubQuery.test_31_calculate_player_percentage_from_players) ... ok
test_32_find_highest_ppg_player_from_nba (__main__.TestSubQuery.test_32_find_highest_ppg_player_from_nba) ... ERROR
test_33_find_brooklyn_nets_roster_from_nba (__main__.TestSubQuery.test_33_find_brooklyn_nets_roster_from_nba) ... ERROR
test_34_calculate_president_votes_percentage_from_twelection2020 (__main__.TestSubQuery.test_34_calculate_president_votes_percentage_from_twelection2020) ... ERROR

ERROR: test_32_find_highest_ppg_player_from_nba (__main__.TestSubQuery.test_32_find_highest_ppg_player_from_nba)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipython-input-1327913058.py", line 16, in test_32_find_highest_ppg_player_from_nba
    highest_ppg_player_from_nba = pd.r

In [18]:
print("您在「{}」章節中的 {} 道 SQL 練習答對了 {} 題。".format(chapter_name, number_of_test_runs, number_of_successes))

您在「子查詢」章節中的 5 道 SQL 練習答對了 2 題。
